In [1]:
import pandas as pd
import numpy as np
import ast
import json
from tqdm import tqdm

In [2]:
df_win = pd.read_parquet("./ultrafeedback.parquet")
print(df_win.shape)

(157675, 8)


In [3]:
df_win.head()

,source,prompt,chosen,chosen-rating,chosen-model,rejected,rejected-rating,rejected-model
0,evol_instruct,Can you write a C++ program that prompts the u...,[{'content': 'Can you write a C++ program that...,5.00,starchat,[{'content': 'Can you write a C++ program that...,1.25,alpaca-7b
1,evol_instruct,Can you write a C++ program that prompts the u...,[{'content': 'Can you write a C++ program that...,5.00,starchat,[{'content': 'Can you write a C++ program that...,4.00,vicuna-33b
2,evol_instruct,Can you write a C++ program that prompts the u...,[{'content': 'Can you write a C++ program that...,5.00,starchat,[{'content': 'Can you write a C++ program that...,1.25,pythia-12b
3,evol_instruct,Suppose you are a content creator and want to ...,[{'content': 'Suppose you are a content creato...,4.75,gpt-4,[{'content': 'Suppose you are a content creato...,4.25,llama-2-13b-chat
4,evol_instruct,Suppose you are a content creator and want to ...,[{'content': 'Suppose you are a content creato...,4.75,gpt-4,[{'content': 'Suppose you are a content creato...,3.25,starchat


In [4]:
df_win["difference"] = df_win["chosen-rating"] - df_win["rejected-rating"]
df_win.drop(columns=["source", "chosen-rating", "rejected-rating"], inplace=True)
df_win.rename(columns={"chosen-model": "model_a", "rejected-model": "model_b"}, inplace=True)
df_win["winner_model_a"] = 1
df_win["winner_model_b"] = 0
df_win["winner_tie"] = 0
df_win.head()

,prompt,chosen,model_a,rejected,model_b,difference,winner_model_a,winner_model_b,winner_tie
0,Can you write a C++ program that prompts the u...,[{'content': 'Can you write a C++ program that...,starchat,[{'content': 'Can you write a C++ program that...,alpaca-7b,3.75,1,0,0
1,Can you write a C++ program that prompts the u...,[{'content': 'Can you write a C++ program that...,starchat,[{'content': 'Can you write a C++ program that...,vicuna-33b,1.00,1,0,0
2,Can you write a C++ program that prompts the u...,[{'content': 'Can you write a C++ program that...,starchat,[{'content': 'Can you write a C++ program that...,pythia-12b,3.75,1,0,0
3,Suppose you are a content creator and want to ...,[{'content': 'Suppose you are a content creato...,gpt-4,[{'content': 'Suppose you are a content creato...,llama-2-13b-chat,0.50,1,0,0
4,Suppose you are a content creator and want to ...,[{'content': 'Suppose you are a content creato...,gpt-4,[{'content': 'Suppose you are a content creato...,starchat,1.50,1,0,0


In [5]:
df_win["chosen"][0]

array([{'content': 'Can you write a C++ program that prompts the user to enter the name of a country and checks if it borders the Mediterranean Sea? Here\'s some starter code to help you out:\n#include <iostream>\n#include <string>\nusing namespace std;\nint main() {\n    string country;\n    // prompt user for input\n    cout << "Enter the name of a country: ";\n    cin >> country;\n    // check if country borders the Mediterranean Sea\n    // [C++ code]\n    return 0;\n}', 'role': 'user'},
       {'content': 'Here\'s a C++ program that prompts the user to enter the name of a country and checks if it borders the Mediterranean Sea:\n\n#include <iostream>\n#include <string>\n#include <set>\n#include <map>\n#include <algorithm>\n\nusing namespace std;\n\nint main() {\n    // store countries and their bordering seas in a map\n    map<string, set<string>> countries;\n    countries["Algeria"] = {"Mediterranean Sea", "North African Coast"};\n    countries["France"] = {"Mediterranean Sea", "E

In [6]:
prompts = []
response_as = []
response_bs = []

for idx, row in tqdm(df_win.iterrows(), total=len(df_win)):
    conversation_a = row["chosen"]
    conversation_b = row["rejected"]

    prompt_a = []
    prompt_b = []
    response_a = []
    response_b = []

    if len(conversation_a) != len(conversation_b):
        print(f"Error: conversation_a and conversation_b have different lengths at index {idx}")
    else:
        for i in range(len(conversation_a)):
            single_round_a = conversation_a[i]
            single_round_b = conversation_b[i]

            if single_round_a["role"] == "user":
                prompt_a.append(single_round_a["content"])
            elif single_round_a["role"] == "assistant":
                response_a.append(single_round_a["content"])
            else:
                print(f"Error: unexpected role at index {idx}")

            if single_round_b["role"] == "user":
                prompt_b.append(single_round_b["content"])
            elif single_round_b["role"] == "assistant":
                response_b.append(single_round_b["content"])
            else:
                print(f"Error: unexpected role at index {idx}")

        if prompt_a != prompt_b:
            print(f"Error: prompt_a and prompt_b have different lengths at index {idx}")
        else:
            prompts.append(prompt_a)
            response_as.append(response_a)
            response_bs.append(response_b)

100%|██████████| 157675/157675 [00:12<00:00, 12626.60it/s]


In [7]:
df_win["prompt"] = prompts
df_win["response_a"] = response_as
df_win["response_b"] = response_bs
df_win.head()

,prompt,chosen,model_a,rejected,model_b,difference,winner_model_a,winner_model_b,winner_tie,response_a,response_b
0,[Can you write a C++ program that prompts the ...,[{'content': 'Can you write a C++ program that...,starchat,[{'content': 'Can you write a C++ program that...,alpaca-7b,3.75,1,0,0,[Here's a C++ program that prompts the user to...,[int main() {\n string country;\n // pro...
1,[Can you write a C++ program that prompts the ...,[{'content': 'Can you write a C++ program that...,starchat,[{'content': 'Can you write a C++ program that...,vicuna-33b,1.00,1,0,0,[Here's a C++ program that prompts the user to...,[Sure! Here's a C++ program that prompts the u...
2,[Can you write a C++ program that prompts the ...,[{'content': 'Can you write a C++ program that...,starchat,[{'content': 'Can you write a C++ program that...,pythia-12b,3.75,1,0,0,[Here's a C++ program that prompts the user to...,"[Sure, here is the program using the C++11 alg..."
3,[Suppose you are a content creator and want to...,[{'content': 'Suppose you are a content creato...,gpt-4,[{'content': 'Suppose you are a content creato...,llama-2-13b-chat,0.50,1,0,0,[To use GPT for generating compelling titles a...,"[Sure, I'd be happy to help! Here's an example..."
4,[Suppose you are a content creator and want to...,[{'content': 'Suppose you are a content creato...,gpt-4,[{'content': 'Suppose you are a content creato...,starchat,1.50,1,0,0,[To use GPT for generating compelling titles a...,[Sure! Although GPT cannot generate text on it...


In [8]:
df_win.drop(columns=["chosen", "rejected"], inplace=True)
df_win.head()

,prompt,model_a,model_b,difference,winner_model_a,winner_model_b,winner_tie,response_a,response_b
0,[Can you write a C++ program that prompts the ...,starchat,alpaca-7b,3.75,1,0,0,[Here's a C++ program that prompts the user to...,[int main() {\n string country;\n // pro...
1,[Can you write a C++ program that prompts the ...,starchat,vicuna-33b,1.00,1,0,0,[Here's a C++ program that prompts the user to...,[Sure! Here's a C++ program that prompts the u...
2,[Can you write a C++ program that prompts the ...,starchat,pythia-12b,3.75,1,0,0,[Here's a C++ program that prompts the user to...,"[Sure, here is the program using the C++11 alg..."
3,[Suppose you are a content creator and want to...,gpt-4,llama-2-13b-chat,0.50,1,0,0,[To use GPT for generating compelling titles a...,"[Sure, I'd be happy to help! Here's an example..."
4,[Suppose you are a content creator and want to...,gpt-4,starchat,1.50,1,0,0,[To use GPT for generating compelling titles a...,[Sure! Although GPT cannot generate text on it...


In [9]:
def swap_columns(row):
    if np.random.rand() < 0.5:
        model_a_value = row["model_a"]
        row["model_a"] = row["model_b"]
        row["model_b"] = model_a_value

        response_a_value = row["response_a"]
        row["response_a"] = row["response_b"]
        row["response_b"] = response_a_value

        winner_model_a_value = row["winner_model_a"]
        row["winner_model_a"] = row["winner_model_b"]
        row["winner_model_b"] = winner_model_a_value
    
    return row

In [10]:
df_win = df_win.apply(swap_columns, axis=1)
df_win.head()

,prompt,model_a,model_b,difference,winner_model_a,winner_model_b,winner_tie,response_a,response_b
0,[Can you write a C++ program that prompts the ...,starchat,alpaca-7b,3.75,1,0,0,[Here's a C++ program that prompts the user to...,[int main() {\n string country;\n // pro...
1,[Can you write a C++ program that prompts the ...,vicuna-33b,starchat,1.00,0,1,0,[Sure! Here's a C++ program that prompts the u...,[Here's a C++ program that prompts the user to...
2,[Can you write a C++ program that prompts the ...,pythia-12b,starchat,3.75,0,1,0,"[Sure, here is the program using the C++11 alg...",[Here's a C++ program that prompts the user to...
3,[Suppose you are a content creator and want to...,llama-2-13b-chat,gpt-4,0.50,0,1,0,"[Sure, I'd be happy to help! Here's an example...",[To use GPT for generating compelling titles a...
4,[Suppose you are a content creator and want to...,starchat,gpt-4,1.50,0,1,0,[Sure! Although GPT cannot generate text on it...,[To use GPT for generating compelling titles a...


In [11]:
df_win.to_csv("kaggle-ultrafeedback.csv", index=False)

In [12]:
df_win = pd.read_csv("kaggle-ultrafeedback.csv")
df_win.head()

,prompt,model_a,model_b,difference,winner_model_a,winner_model_b,winner_tie,response_a,response_b
0,['Can you write a C++ program that prompts the...,starchat,alpaca-7b,3.75,1,0,0,['Here\'s a C++ program that prompts the user ...,['int main() {\n string country;\n // pr...
1,['Can you write a C++ program that prompts the...,vicuna-33b,starchat,1.00,0,1,0,['Sure! Here\'s a C++ program that prompts the...,['Here\'s a C++ program that prompts the user ...
2,['Can you write a C++ program that prompts the...,pythia-12b,starchat,3.75,0,1,0,"['Sure, here is the program using the C++11 al...",['Here\'s a C++ program that prompts the user ...
3,['Suppose you are a content creator and want t...,llama-2-13b-chat,gpt-4,0.50,0,1,0,"['Sure, I\'d be happy to help! Here\'s an exam...",['To use GPT for generating compelling titles ...
4,['Suppose you are a content creator and want t...,starchat,gpt-4,1.50,0,1,0,"[""Sure! Although GPT cannot generate text on i...",['To use GPT for generating compelling titles ...


In [13]:
def ensure_json_loads(input_str):
    python_list = ast.literal_eval(input_str)
    return json.dumps(python_list, ensure_ascii=False)

df_win.loc[:, "prompt"] = df_win["prompt"].apply(ensure_json_loads)
df_win.loc[:, "response_a"] = df_win["response_a"].apply(ensure_json_loads)
df_win.loc[:, "response_b"] = df_win["response_b"].apply(ensure_json_loads)
df_win.head()

,prompt,model_a,model_b,difference,winner_model_a,winner_model_b,winner_tie,response_a,response_b
0,"[""Can you write a C++ program that prompts the...",starchat,alpaca-7b,3.75,1,0,0,"[""Here's a C++ program that prompts the user t...","[""int main() {\n string country;\n // pr..."
1,"[""Can you write a C++ program that prompts the...",vicuna-33b,starchat,1.00,0,1,0,"[""Sure! Here's a C++ program that prompts the ...","[""Here's a C++ program that prompts the user t..."
2,"[""Can you write a C++ program that prompts the...",pythia-12b,starchat,3.75,0,1,0,"[""Sure, here is the program using the C++11 al...","[""Here's a C++ program that prompts the user t..."
3,"[""Suppose you are a content creator and want t...",llama-2-13b-chat,gpt-4,0.50,0,1,0,"[""Sure, I'd be happy to help! Here's an exampl...","[""To use GPT for generating compelling titles ..."
4,"[""Suppose you are a content creator and want t...",starchat,gpt-4,1.50,0,1,0,"[""Sure! Although GPT cannot generate text on i...","[""To use GPT for generating compelling titles ..."


In [14]:
df_win_drop_duplicated_prompts = df_win.loc[df_win.groupby("prompt")["difference"].idxmax()]
print(df_win_drop_duplicated_prompts.shape)
df_win_drop_duplicated_prompts.reset_index(drop=True, inplace=True)
df_win_drop_duplicated_prompts.head()

(60906, 9)


,prompt,model_a,model_b,difference,winner_model_a,winner_model_b,winner_tie,response_a,response_b
0,"["" Given a sentence in the English language, t...",llama-2-70b-chat,vicuna-33b,1.25,1,0,0,"[""I don't think that's an appropriate or respe...","[""वे भी क्रमिक त्वचा, निविधता स्वरूप अँवलोकन र..."
1,"["" Given a sentence in the English language, t...",gpt-4,llama-2-13b-chat,4.00,1,0,0,"[""ठूलो मास भएको ताराको चमक, धेरै बढी हुन्छ, र ...","[""Sure, I'd be happy to help! Here's the trans..."
2,"["" Given a sentence in the English language, t...",starchat,alpaca-7b,2.00,1,0,0,"[""In the Nepali language, the sentence would b...","[""ये लाग्दे खोयेचे पीचोगियु र घोख्ने बजारोबोन ..."
3,"["" In this task, you are given a sentence in t...",gpt-4,alpaca-7b,3.75,1,0,0,"[""Σε σταθερή κατάσταση, το ντεχυδρο-αριπιπραζό...","[""Όλα μου θέλω να ξέρετε την αρχή και την επόμ..."
4,"["" In this task, you are given a sentence in t...",falcon-40b-instruct,gpt-3.5-turbo,2.50,0,1,0,"[""No questions were asked in the given input.""]","[""6.2 Ασυμβατότητες\n\nConfidence: 95%""]"


In [15]:
print(df_win_drop_duplicated_prompts["prompt"][0])
print(df_win_drop_duplicated_prompts["prompt"][1])

[" Given a sentence in the English language, translate it into the Nepali language.\nOne example is below.\nQ: Text: If the research problem is not clear then review of literature can help focus on the problem.\n\nA: अथवा शोध समस्याबारे स्पष्ट हुन सकिरहेको छैन भने साहित्यको अवलोकन र पुनरावलोकनले ध्यान केन्द्रित गर्न सहयोग गर्छ ।\n\nRationale: The translation from English to Nepali is correct\nQ: Text: They can also discover comets, and perform regular observations of variable stars.\n\nA:"]
[" Given a sentence in the English language, translate it into the Nepali language.\nOne example: Text: If the research problem is not clear then review of literature can help focus on the problem.\n\nSolution is here: अथवा शोध समस्याबारे स्पष्ट हुन सकिरहेको छैन भने साहित्यको अवलोकन र पुनरावलोकनले ध्यान केन्द्रित गर्न सहयोग गर्छ ।\n\nExplanation: The translation from English to Nepali is correct\n\nNow, solve this: Text: The more massive the star, the greater its luminosity, and the more rapidly it 

In [16]:
df_win_drop_duplicated_prompts.to_csv("kaggle-ultrafeedback-drop-duplicate.csv", index=False)
df_win.to_csv("kaggle-ultrafeedback.csv", index=False)

In [17]:
def unicode_preprocess(input_str):
    return json.loads(input_str)

df_win.loc[:, "prompt"] = df_win["prompt"].apply(unicode_preprocess)
df_win.loc[:, "response_a"] = df_win["response_a"].apply(unicode_preprocess)
df_win.loc[:, "response_b"] = df_win["response_b"].apply(unicode_preprocess)

In [18]:
type(df_win["response_a"][0])

list